In [13]:
from mlflow.tracking import MlflowClient

TRACKING_URI="sqlite:///mlflow.db"

client=MlflowClient(tracking_uri=TRACKING_URI)

In [14]:
from mlflow.entities import ViewType


all_runs=client.search_runs(experiment_ids=1,
                            filter_string="",
                            run_view_type=ViewType.ALL,
                            max_results=10,order_by=["metrics.rmse ASC"])

In [15]:
for run in all_runs:
    print(f"run id : {run.info.run_id} rmse : {run.data.metrics['rmse']:.4f}")

run id : df2d51c4c2f641fc9c29d65dccdb5238 rmse : 6.0908
run id : 6d9f2c4747354b90888158ec59dd82c6 rmse : 6.0942
run id : bb34ed8d2eab4ae6b9ebefd856ce5f8f rmse : 6.0945
run id : 75a661fc9c214765bce5174c79009e19 rmse : 6.0946
run id : 21db4852cd514ed99dc9a5c142470c55 rmse : 6.0950
run id : 3971ff5e627b465496931789c8d5226e rmse : 6.0957
run id : 532a40698f47402fb631a2c2a794ad8a rmse : 6.0958
run id : cf057cfdd1a04065ae08453928e96e90 rmse : 6.0960
run id : 637e594f956a4e4b8cad2fa78be5bc15 rmse : 6.0961
run id : 88305023e0194871b7c371b08ca8ceb0 rmse : 6.0994


In [16]:
import mlflow


mlflow.set_tracking_uri(TRACKING_URI)

In [9]:
run_id="8ffca8da5e234482a027dcacfc691cce"

model_uri=f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri,name="nyc-models")

Registered model 'nyc-models' already exists. Creating a new version of this model...
Created version '9' of model 'nyc-models'.


<ModelVersion: aliases=[], creation_timestamp=1725110675134, current_stage='None', description=None, last_updated_timestamp=1725110675134, name='nyc-models', run_id='8ffca8da5e234482a027dcacfc691cce', run_link=None, source='/workspaces/MLOps-DataTalksClub/02_experiment_tracking/mlruns/1/8ffca8da5e234482a027dcacfc691cce/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=9>

In [22]:
model_name="nyc-models"

latest_versions=client.get_latest_versions(name=model_name)

/tmp/ipykernel_3743/3751025032.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions=client.get_latest_versions(name=model_name)


In [23]:
for version in latest_versions:
    print(f"version : {version.version} stage: {version.current_stage}")

version : 9 stage: Staging


In [24]:
model_version=4
new_stage="Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False)

/tmp/ipykernel_7057/1147022751.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1725086044555, current_stage='Staging', description='This model version 4 was transitioned to Staging on 2024-08-31', last_updated_timestamp=1725104427324, name='nyc-models', run_id='df2d51c4c2f641fc9c29d65dccdb5238', run_link=None, source='/workspaces/MLOps-DataTalksClub/02_experiment_tracking/mlruns/1/df2d51c4c2f641fc9c29d65dccdb5238/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [25]:
from datetime import datetime

date=datetime.today().date()

client.update_model_version(
    name=model_name,
    version=4,
    description=f"This model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1725086044555, current_stage='Staging', description='This model version 4 was transitioned to Staging on 2024-08-31', last_updated_timestamp=1725104430183, name='nyc-models', run_id='df2d51c4c2f641fc9c29d65dccdb5238', run_link=None, source='/workspaces/MLOps-DataTalksClub/02_experiment_tracking/mlruns/1/df2d51c4c2f641fc9c29d65dccdb5238/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [1]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [12]:
df = pd.read_parquet("data/green_tripdata_2021-03.parquet")

In [27]:
client.download_artifacts(run_id=run_id,path="models/preprocessor.b",dst_path='.')

MlflowException: The following failures occurred while downloading one or more artifacts from /workspaces/MLOps-DataTalksClub/02_experiment_tracking/mlruns/1/df2d51c4c2f641fc9c29d65dccdb5238/artifacts:
##### File models/preprocessor.b #####
[Errno 2] No such file or directory: '/workspaces/MLOps-DataTalksClub/02_experiment_tracking/mlruns/1/df2d51c4c2f641fc9c29d65dccdb5238/artifacts/models/preprocessor.b'

In [16]:
run_id

'df2d51c4c2f641fc9c29d65dccdb5238'